- 예제 소스코드 및 데이터 : https://github.com/gilbutITbook/006975

## 딥러닝이란?
- 머신러닝의 특정한 한 분야로서 연속된 층(layer)에서 점진적으로 의미있는 표현을 학습하는 방식
<img src = 'https://thebook.io/img/006975/035.jpg' width ='40%' height='40%'/>
<br><br>

- 층에서 입력데이터가 처리되는 상세 내용은 가중치(weight)에 저장되어 있다.<br>어떤 층에서 일어나는 변환은 그 층의 가중치를 파라미터로 가지는 함수로 표현된다.(가중치를 그 층의 파라미터라고도 함)
- 즉, 딥러닝 학습은 주어진 입력을 정확한 타깃에 매핑하기 위해 신경망의 모든 층에 있는 가중치 값을 찾는 것이다.
<img src = 'https://thebook.io/img/006975/036_1.jpg' width ='40%' height='40%'/>
<br><br>

- 정확한 값을 찾는 것은 매우 어렵다. 파라미터 하나의 값을 바꾸면 모든 파라미터에 영향을 끼치기 때문이다.
- 신경망 출력을 제어하기 위해서는 손실함수(loss function)로 출력이 기댓값에서 얼마나 벗어났는지 측정해야 한다.<br>신경망 예측을 측정하기 위해 손실함수가 신경망의 예측과 기대하는 진짜 타깃의 차이를 점수로 계산한다.<br>(손실함수는 목적함수, 비용함수라고도 불린다)
<img src = 'https://thebook.io/img/006975/036_2.jpg' width ='40%' height='40%'/>
<br><br>

- 딥러닝 방식은 위 예측 점수를 피드백 신호로 사용하여 손실 점수가 감소되는 방향으로 가중치 값을 조금씩 수정하는 것이다.
- 수정과정은 역전파 알고리즘을 구현한 옵티마이저(optimizer)가 담당한다.<br>초기에는 랜덤한 가중치로 변환하여 수정하면서 훈련 반복을 통해 점진적으로 올바른 방향으로 조정되고 손실 점수가 감소한다.<br>충분한 횟수만큼 반복하면 손실함수를 최소화하는 가중치 값을 산출한다.
<img src = 'https://thebook.io/img/006975/037.jpg' width ='40%' height='40%'/>
<br><br>

## 신경망의 수학적 구성 요소
### 1. 신경망 맛보기
- Dense : 신경망층
- optimizer : 옵티마이저, 입력된 데이터와 손실함수 기반으로 네트워크 업데이트
- loss : 손실함수, 훈련데이터에서 신경망 성능을 측정하는 방법으로 네트워크가 옳은 방향으로 학습될 수 있도록 도움
<br><br>
- 옵티마이저는 손실에 대한 그래디언트가 파라미터를 업데이트하는 정확한 방식으로<br>RMSProp 옵티마이저와 모멘텀을 사용한 SGD 등이 있다.

In [7]:
## 신경망 훈련 준비 -----------------------------------------------------------
# 1) 데이터셋 구성
from keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(train_images.shape, train_labels.shape, test_images.shape, test_labels.shape)

# 2) 신경망 구성
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28*28, )))
network.add(layers.Dense(10, activation='softmax'))

# 3) 컴파일 단계
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [8]:
## 데이터 전처리 -----------------------------------------------------------
# 1) 이미지데이터 one-hot 인코딩 스케일 조정
train_images = train_images.reshape((60000, 28*28))
train_images = train_images.astype('float32')/255

test_images = test_images.reshape((10000, 28*28))
test_images = test_images.astype('float32')/255

# 2) 레이블데이터 범주형 인코딩
from keras.utils.np_utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [9]:
## 신경망 훈련 -----------------------------------------------------------
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 2s 4ms/step - loss: 0.2589 - accuracy: 0.9253
Epoch 2/5
469/469 [==============================] - 2s 4ms/step - loss: 0.1034 - accuracy: 0.9696
Epoch 3/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0674 - accuracy: 0.9798
Epoch 4/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0492 - accuracy: 0.9850
Epoch 5/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0369 - accuracy: 0.9890


In [11]:
test_loss, test_acc = network.evaluate(test_images, test_labels)
print('test_loss :', test_loss)
print('test_acc :', test_acc)

# 훈련정확도 0.989 > 테스트 정확도 0.979

313/313 [==============================] - 1s 3ms/step - loss: 0.0691 - accuracy: 0.9788
test_loss : 0.06907019019126892
test_acc : 0.9787999987602234


### 2. 데이터 표현
- 텐서(tensor) : 머신러닝 기본 데이터 구조, 다차원 넘파이 배열로 차원(demension)을 축(axis)이라고 부른다.
- <b>스칼라(0차원 텐서)</b> : 하나의 숫자만 담고 있는 텐서
- <b>벡터(1차원 텐서)</b> : 숫자의 배열
- <b>행렬(2차원 텐서)</b> : 벡터의 배열
- 행렬의 크기가 (3,5)이고 3차원 텐서의 크기는 (3, 3, 5)

- ndim 속성 : 넘파이 배열 축 개수(랭크; rank) 확인

In [13]:
import numpy as np

## 1) 스칼라(0D 텐서)
x0 = np.array(12)
print('스칼라 :', x0)
print('rank :', x0.ndim)

## 2) 벡터(1D 텐서)
x1 = np.array([12, 3, 6, 14, 7])
print('벡터 :', x1)
print('rank :', x1.ndim)

## 3) 행렬(2D 텐서)
x2 = np.array([12, 3, 6, 14, 7])
print('행렬 :', x2)
print('rank :', x2.ndim)

스칼라 : 12
rank : 0
벡터 : [12  3  6 14  7]
rank : 1
행렬 : [12  3  6 14  7]
rank : 1


In [25]:
## 4) 넘파이 텐서 조작
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

ex_tensor = train_images[:100]
print(ex_tensor.shape)

ex_tensor = train_images[:100, :, :]
print(ex_tensor.shape)

ex_tensor = train_images[:100, :28, :28]
print(ex_tensor.shape)

(100, 28, 28)
(100, 28, 28)
(100, 28, 28)


In [26]:
## 5) 텐서 크기 변환 : 신경망에 주입할 숫자 데이터 전처리에 사용
x = np.zeros((300, 20))
print(x.shape)

x = x.reshape((100,60))
print(x.shape)

# 전치(transposition)
x = np.zeros((300, 20))
x = np.transpose(x)
print(x.shape)

(300, 20)
(100, 60)
(20, 300)


### 3. 신경망 구조
#### 1) <b>층(Layer)</b>
- 층(layer)은 딥러닝의 구성 단위로 하나 이상의 텐서를 입력받아 하나 이상의 텐서를 출력하는 데이터 처리 모듈이다.
- 대부분의 경우 가중치를 가지며 이때 가중치는 층의 상태를 말한다.<br>가중치는 확률적 경사 하강법으로 학습되는 하나 이상의 텐서로 네트워크가 학습한 지식이 담겨있다.
- 층마다 적절한 텐서와 데이터 처리방식이 다르다.<br>
2D 텐서가 저장된 간단한 벡터 데이터는 완전연결층(fully connected), 밀집층(dense)<br>
3D 텐서로 저장된 시퀀스 데이터는 LSTM 같은 순환층(recurrent)<br>
4D 텐서로 저장된 이미지 데이터는 2D 합성곱층(convolution)에 의해 처리된다.

#### 2) <b>모델</b>
- 모델은 층의 네트워크로 하나의 입력을 하나의 출력으로 매핑하는 층을 순서대로 쌓는 것이다.

#### 3) <b>손실함수와 옵티마이저</b>
- 손실함수 : 훈련하는 동안 최소화될 값으로 주어진 문제에 대한 성공지표가 된다.
- 옵티마이저 : 손실함수를 기반으로 네트워크가 어떻게 업데이트될지 결정하며, <br>특정 종류의 확률적 경사 하강법(SGD)을 구현한다.